# Abstract

In this tutorial we introduce the "Bayes by Backprop" algorithm. It is an algorithm to make the calculation of the posterior weight distribution of neural networks tractable. The main idea of this method is to combine two standard methods, one from the Bayesian real and one from the frequentist realm:

1. Variation inference: this is a standard technique to approximate the posterior distribution in situations that are otherwise intractable. The fundamental idea is very simple: we can't do the posterior calculations (or sampling, for that matter) exactly, so we approximate the posterior with distributions we know.
2. Backpropagation (a.k.a. gradient descent): this is the default technique to optimize the weights in neural networks.

It is shown that we can use backpropagation to perform variational inference.

We use "Bayes by Backprop" on three different datasets:

1. A simulated dataset for a classification task with different Bayes rate in different regions.
2. MNIST: here we reproduce the results from the original paper and extend the method to convolutional neural networks (*at least, we try?*)
3. TBD



# Introduction

## What are Bayesian neural networks

## Why Bayesian nets

## Problems with naive approach (sampling)

## Solution with Variation inference

# Variational inference

## What is variational inference?

## Easy example (maybe Gaussian and Gaussian mixture)

# Bayes by backprop

## Derivation

## Variational posterior

## Description of prior

## Gradient descent

# Application and Implementation

## Simulated dataset

Possibly binary classification with different best achievable accuracy in different regions.

## MNIST

Could try to extend algorithm to CNN. Should not be too hard (don't blame me for that sentence later)

## Real life dataset

(Maybe claims frequency prediction)

# Summary

# References

+ https://gluon.mxnet.io/chapter18_variational-methods-and-uncertainty/bayes-by-backprop.html
+ https://arxiv.org/pdf/1505.05424.pdf